In [20]:
#import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from requests import get

### Documentation of the api used to fetch altitudes: https://github.com/Jorl17/open-elevation

In [32]:
#load train data
train_data=pd.read_csv('habitat_locs.csv',index_col=[0]) 


In [89]:
#get the training Florida data

#Keep coordinates that are within florida boundaries
habitats_fl_temp = train_data[ (train_data['Lat'] >= 24.27) & (train_data['Lat'] <= 31.20)]
habitats_fl = habitats_fl_temp[ (habitats_fl_temp['Lon'] <= -80.02) & (habitats_fl_temp['Lon'] >= -87.38)]
train_fl=habitats_fl.copy()


In [38]:
len(train_fl)

3546

In [33]:
train_data

,Lat,Lon,Cls,species_present
0,-18.286728,143.481247,Aw,31529
1,-13.099798,130.783646,Aw,31529
2,-13.965274,131.695145,Aw,31529
3,-12.853950,132.800507,Aw,31529
4,-12.196790,134.279327,Aw,31529
...,...,...,...,...
272032,33.716885,73.203621,Cwa,145031
272033,24.600239,72.730560,BSh,145031
272034,18.849600,80.654129,Aw,145031
272035,21.073837,75.945656,BSh,145031


In [39]:
# script for returning elevation from lat, long, based on open elevation data
# which in turn is based on SRTM
def get_elevation(lat = None, long = None):
    '''
        A custom script designed to retrieve elevation data in meters based on latitude and longitude coordinates.
    '''
    if lat is None or long is None: return None
    
    query = ('https://api.open-elevation.com/api/v1/lookup'
             f'?locations={lat},{long}')
    
    # Making a request with a 20-second timeout limit for sluggish responses
    r = get(query, timeout = 20)

    # Parsing the JSON response only for status codes 200 or 201
    if r.status_code == 200 or r.status_code == 201:
        elevation = pd.json_normalize(r.json(), 'results')['elevation'].values[0]
    else: 
        elevation = None
    return elevation

In [40]:
#Example of altitude fetching:
print(f'First coordinates: {train_data["Lat"][0]} , {train_data["Lon"][0]}.')
print(f'First altitude: {get_elevation(train_data["Lat"][0],train_data["Lon"][0])}')

print(f'Second coordinates: {train_data["Lat"][1]} , {train_data["Lon"][1]}.')
print(f'Second altitude: {get_elevation(train_data["Lat"][1],train_data["Lon"][1])}')

First coordinates: -18.28672790527344 , 143.4812469482422.
First altitude: 295.0
Second coordinates: -13.099798202514648 , 130.7836456298828.
Second altitude: 98.0


In [63]:
#Get a full list of altitudes from the Florida training dataset
ids=train_fl.index.tolist()
fl_altitudes=[]


for i in tqdm(ids,position=0, leave=True):
    lat,lon=train_fl['Lat'][i],train_fl['Lon'][i]
    alt=get_elevation(lat,lon)
    fl_altitudes.append(alt)
   

100%|██████████| 3546/3546 [07:15<00:00,  8.14it/s]


In [79]:
#retry for empty indicies
empty_fl=[i for i in range(len(fl_altitudes)) if fl_altitudes[i] == None]
print(f'{len(empty_fl)} gaps pending...\n')

for i in empty_fl:
    locate=ids[i]
    lat,lon=train_fl['Lat'][locate],train_fl['Lon'][locate]
    alt=get_elevation(lat,lon)
    fl_altitudes[i]=alt
    if fl_altitudes[i]!=None:
        print(f'Succesful replacement at id {i}.')
    else:
        print(f'Failed replacement at id {i}.')

2 gaps pending...

Failed replacement at id 667.
Failed replacement at id 668.


In [87]:
#if some coordinates are consistently not registered, test out of loop, leave be if None is returned
print(f'Coordinates: {train_data["Lat"][ids[668]]} , {train_data["Lon"][ids[668]]}.')
print(f'Altitude: {get_elevation(train_data["Lat"][ids[668]],train_data["Lon"][ids[668]])}')


Coordinates: 27.492679595947266 , -80.95001220703125.
Altitude: None


In [90]:
#update the dataset
train_fl['Alt']=fl_altitudes


In [92]:
#save the data
train_fl.to_csv('habitats_fl.csv')